# MushCamp EfficientNetB5

### Import Library

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay

from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from efficientnet.tfkeras import EfficientNetB5

### Load Data

In [ ]:
trainPath = r'dataset\dataSplit\train'
validationPath = r'dataset\dataSplit\val'
testPath = r'dataset\dataSplit\test'

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

### Data Augmentation

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=0.45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(
    rescale=1./255
)

#Train
train_generator = train_datagen.flow_from_directory(
                                            trainPath,
                                                    target_size=(300, 300),
                                                    batch_size=64,
                                                    shuffle=True,
                                                    color_mode='rgb',
                                                    class_mode='categorical',
                                            )

#Validation
validation_generator=val_datagen.flow_from_directory(
                                            validationPath,
                                                    target_size=(300,300),
                                                    batch_size=64,
                                                    shuffle=True,
                                                    color_mode='rgb',
                                                    class_mode='categorical',
                                                )

#Test
test_generator=val_datagen.flow_from_directory(
                                            testPath,
                                                    target_size=(300,300),
                                                    batch_size=64,
                                                    shuffle=True,
                                                    color_mode='rgb',
                                                    class_mode='categorical',
                                                )

##### Load Model EfficientNetB5

In [ ]:
pre_trained_model_B5 = EfficientNetB5(
    include_top=False,
    weights="imagenet",
    input_shape=(300, 300, 3),
    pooling='avg',
)

##### layering EfficientNetB5

In [ ]:
for layer in pre_trained_model.layers:
    layer.trainable = False


last_layer = pre_trained_model.get_layer('avg_pool')
last_output = last_layer.output


x = layers.Flatten()(last_output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(1024, activation='relu')(x)              
x = layers.Dense (9, activation='softmax')(x)           

model_B5 = Model( pre_trained_model_B5.input, x) 

model_B5.compile(optimizer = 'adam', 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

##### Training Model with EfficientNetB5

In [ ]:
history = model_B5.fit(
            train_generator,
            validation_data = validation_generator,
            epochs = 20,
            callbacks=[reduceLROnPlat],
            verbose = 1)  

##### Evaluating Model using Data Testing

In [ ]:
B5_eval = model_B5.evaluate(test_generator,verbose=0)
B5_acc = round(B5_eval[1],2) * 100
B5_acc

##### Save Model EfficientNetB5

In [ ]:
model_B5.save('saved-models/model_EfficientNetB5.h5')

##### Confusion Matrix B5

In [ ]:
y_test = []
y_pred = []

for i in range(len(test_generator)) :
    x,y = test_generator.next()
    for j in range(len(y)) :
        y_test.append(class_names[tf.argmax(y[j])])
        y_pred.append(class_names[np.argmax(model_B5.predict(x[j][None,...],verbose=0))])

In [ ]:
fig, ax = plt.subplots(figsize=(18, 13),dpi=150)
cm = confusion_matrix(y_test, y_pred)

colors = ["#393E46","#393E46"]
colormap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)

ax.text(0,-0.3,'Confusion Matrix ',fontfamily='serif',fontsize=15,fontweight='bold')
ax.text(0,-0.12,'EfficientB7 Model',fontfamily='serif',fontsize=12,fontweight='light')
sns.heatmap(ax=ax, data=cm,
            square=True, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=1.5, annot=True,cmap=colormap, 
            annot_kws={"fontsize":12},fmt='')

ax.set_xticklabels(class_names)
ax.set_yticklabels(class_names)

ax.set_ylabel("Actual Label",fontsize=10)
ax.set_xlabel("Predicted Label",fontsize=10)

from matplotlib.patches import Rectangle
for i in range(0,8):
    ax.add_patch(Rectangle((i, i), 1, 1, fill=True,color='#00ADB5'))

plt.show()

##### Prediction Image B5

In [ ]:
from keras.models import load_model
import keras.utils as image
import matplotlib.pyplot as plt
import numpy as np

model = load_model('Saved_Models/model_EfficientNetB5.h5')

In [ ]:
B5_eval = model_B5.evaluate(test_generator,verbose=0)
B5_acc = round(B5_eval[1],2) * 100
B5_acc

In [ ]:
img_path = ""
CLASS_NAMES = ['Agglonema', 'Alocasia', 'Gelombang Cinta', 'Janda Bolong', 'Lidah Mertua', 'Lili Paris', 'Pucuk Merah', 'Suplir']

img = image.load_img(img_path, target_size=(275,275))
imgplot = plt.imshow(img)
x = image.img_to_array(img)
x = x/255.0
x = np.expand_dims(x, axis=0)
images = np.vstack([x])

classes = model.predict(images, batch_size=10,verbose=0)
print(f'Hasil Prediksi: {CLASS_NAMES[np.argmax(classes)]}')